In [1]:
import torch
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    ShortSideScale,
    UniformTemporalSubsample,
    Div255,
)
from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    Resize,
    RandomHorizontalFlip,
)
from torch.utils.data import DataLoader
import torch.nn as nn
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"


%cd /workspace/Walk_Video_PyTorch/project


/workspace/Walk_Video_PyTorch/project


In [2]:
from pytorch_lightning import seed_everything
from torchmetrics.classification import BinaryAccuracy
from models.pytorchvideo_models import WalkVideoClassificationLightningModule
from dataloader.data_loader import WalkDataModule

from pytorch_lightning import Trainer

from IPython.display import clear_output

clear_output()


seed_everything(42, workers=True)


Global seed set to 42


42

In [3]:
from parameters import get_parameters

VIDEO_LENGTH = ['1', '2', '3']
VIDEO_FRAME = ['8', '16', '32']

opt, _ = get_parameters()
opt.num_workers = 8
opt.batch_size = 8
opt.gpu_num = 1

opt.version = '516_1_8'
opt.model = "resnet"
opt.model_depth = 50
opt.model_class_num = 1

# opt.clip_duration = 2
# opt.uniform_temporal_subsample_num = 30
opt.version = opt.version + '_' + opt.model + '_depth' + str(opt.model_depth)

opt.fusion_method = 'slow_fusion'
opt.fix_layer = 'all'
# opt.train_path = '/workspace/data/split_pad_dataset_512/fold3/'

opt.transfor_learning = True
opt.pre_process_flag = True

DATA_PATH = opt.split_pad_data_path


In [4]:
opt.train_path = os.path.join(DATA_PATH, 'flod3')

opt.clip_duration = int(1)
opt.uniform_temporal_subsample_num = int(8)

# ckpt_path = '/workspace/Walk_Video_PyTorch/logs/resnet/1119_1_8_resnet_depth50/flod3/checkpoints/epoch=23-val_loss=0.47-val_acc=0.8568.ckpt'
ckpt_path = '/workspace/Walk_Video_PyTorch/logs/resnet/516_1_8_resnet_depth50/flod0/checkpoints/epoch=17-val_loss=1.82-val_acc=0.7080.ckpt'

print('#' * 50)
print('the data path: %s' % opt.train_path)
print('ckpt: %s' % ckpt_path)
model = WalkVideoClassificationLightningModule(opt).load_from_checkpoint(ckpt_path)

data_module = WalkDataModule(opt)
data_module.setup()
test_data = data_module.val_dataloader()


##################################################
the data path: /workspace/data/split_pad_dataset_512/flod3
ckpt: /workspace/Walk_Video_PyTorch/logs/resnet/516_1_8_resnet_depth50/flod0/checkpoints/epoch=17-val_loss=1.82-val_acc=0.7080.ckpt


Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main
Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


##################################################
run pre process model! /workspace/data/split_pad_dataset_512/flod3
##################################################


In [5]:
transform = Compose(
    [
        ApplyTransformToKey(
            key="video",
            transform=Compose(
                [
                    # uniform clip T frames from the given n sec video.
                    UniformTemporalSubsample(8),

                    # dived the pixel from [0, 255] tp [0, 1], to save computing resources.
                    Div255(),
                    Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),

                    Resize(size=[224, 224]),
                    # RandomHorizontalFlip(p=0.5),
                ]
            ),
        ),
    ]
)


In [6]:
from data_loader import WalkDataset
from pytorchvideo.data import make_clip_sampler

dset = WalkDataset(
    data_path='/workspace/Walk_Video_PyTorch/project/misc/test_video/',
    clip_sampler=make_clip_sampler('uniform', 1),
    video_sampler=torch.utils.data.SequentialSampler,
    transform=transform,
)

test_data = DataLoader(
    dset,
    batch_size=300
)


In [7]:
batch = next(iter(test_data))

video = batch['video'].detach()  # b, c, t, h, w
label = batch['label'].detach()  # b, class num
label = [0] * len(label)
name = batch['video_name']

# video.shape, label, name, len(name)


# Pred the result 
use pre trained model to predict the reulst

In [8]:
# pred the result
model = model.cuda()
video = video.cuda()

with torch.no_grad():
    pred = model(video)

pred = pred.cpu()


In [9]:
pred_list = []

for i in torch.sigmoid(pred):
    if i > 0.5:
        pred_list.append(1)
    else:
        pred_list.append(0)


In [10]:
# pred_list, label,
len(label)


60

In [11]:
# calc acc
acc = BinaryAccuracy(threshold=0.5)

acc(torch.sigmoid(pred).squeeze(), torch.tensor(label))


tensor(0.7167)

In [12]:
acc(torch.sigmoid(pred[:65]).squeeze(), torch.tensor(label[:65]))
for i in range(0, len(label), len(label)//3):
    print(acc(torch.sigmoid(pred[i:i+len(label)//3]).squeeze(), torch.tensor(label[i:i+len(label)//3])))

tensor(0.7500)
tensor(0.7500)
tensor(0.6500)


In [13]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

number_list = []

for i, _ in enumerate(name):
    number = _.split('-')[1].split('.')[0]
    number_list.append(number)

category_names = list(set(number_list))
category_names.sort()

results = {
    '': list(Counter(number_list).values())
}


def survey(results, category_names, pred: list, label: list, video_num: int):
    """
    Parameters
    ----------
    results : dict
        A mapping from question labels to a list of answers per category.
        It is assumed all lists contain the same number of entries and that
        it matches the length of *category_names*.
    category_names : list of str
        The category labels.
    """
    labels = list(results.keys())
    data = np.array(list(results.values()))
    data_cum = data.cumsum(axis=1)
    category_colors = plt.colormaps['RdYlGn'](
        np.linspace(0.15, 0.85, data.shape[1]))

    fig, ax = plt.subplots(nrows=2, gridspec_kw={'height_ratios': [1.5, 0.5]})
    ax[1].invert_yaxis()
    ax[1].set_xlim(0, np.sum(data, axis=1).max())

    for i, (colname, color) in enumerate(zip(category_names, category_colors)):
        print((colname, color))
        widths = data[:, i]
        starts = data_cum[:, i] - widths

        rects = ax[1].barh(labels, widths, left=starts, height=0.1,
                           label=colname, color=color)

        r, g, b, _ = color
        text_color = 'white' if r * g * b < 0.5 else 'darkgrey'

        ax[1].bar_label(rects, label_type='center', color=text_color)

    ax[1].set_title('The number of video')

    # 生成图形
    ax[0].plot(video_number, pred, 'bo:', label='predict label', linewidth=2)  # 颜色绿色，点形圆形，线性虚线，设置图例显示内容，线条宽度为2
    ax[0].plot(video_number, label, 'r.', label='true label', linewidth=1)

    ax[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)  # 显示图例, 图例中内容由 label 定义
    # ax[0].set_title('%s positive rate' % name[0].split('-')[0])  # 图形的标题
    ax[0].set_ylabel('label')
    ax[0].autoscale(enable=True, axis="x", tight=True)

    fig.subplots_adjust(hspace=0.5) # 甚至两个图片中间的距离

    return fig, ax


# 生成数据
video_number = range(0, len(label))
pred = pred_list
label = label

fig, ax = survey(results, category_names, pred, label, video_number)

# ax[0].ylabel('label') # 横坐标轴的标题
# ax[0].xlabel('video number') # 纵坐标轴的标题
# ax[0].xticks(range(0, len(label))) # 设置横坐标轴的刻度为 0 到 10 的数组
# ax[0].ylim([-0.25, 1.25]) # 设置纵坐标轴范围为 -2 到 2

plt.show()


IndexError: list index out of range

In [ ]:
# # delete the start and finish 1s in raw video
# import os 

# from torchvision.io import read_video, write_video
# data_path='/workspace/Walk_Video_PyTorch/project/misc/test_video/'


# def slice_start_end_1s(path):
#     full_path = os.path.join(data_path, 'ASD', path)
#     f, a, info = read_video(full_path, pts_unit='sec')
#     write_video(filename=full_path, video_array=f[30:-30], fps=30)
#     print('write video:%s' % full_path)


# path = os.listdir(data_path+'ASD')
# for p in path:
#     slice_start_end_1s(p)

# Grad CAM pytorch 
use pytorch_grad_cam api to make visilizaiton.

In [ ]:
from pytorch_grad_cam import GradCAM, HiResCAM, FullGrad, GradCAMPlusPlus, AblationCAM, ScoreCAM, LayerCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

# guided grad cam method
target_layer = [model.model.blocks[-2].res_blocks[-1]]
# target_layer = [ model.model.blocks[-2]]

cam = LayerCAM(model, target_layer, use_cuda=True)
# cam = GradCAM(model, target_layer, use_cuda=True)
targets = [ClassifierOutputTarget(-1)]

grayscale_cam = cam(video[:20], aug_smooth=True, eigen_smooth=True)



In [ ]:
from captum.attr import visualization as viz
import numpy as np 

def show_cam(video:torch.Tensor, cam):

    b, h, w, t = grayscale_cam.shape

    for i in range(b):
        cam_map = cam[i].mean(axis=2).squeeze()
        # cam_map = grayscale_cam[:,:,:,0].squeeze()
        cam_map = np.expand_dims(cam_map, 2)

        inp_tensor = video[i,:,1,:].permute(1,2,0).cpu().detach().numpy()

        # use captum visual method
        figure, axis = viz.visualize_image_attr_multiple(
            cam_map,
            inp_tensor,
            methods=['original_image', 'blended_heat_map'],
            signs=['all', 'positive'],
            show_colorbar=True,
            outlier_perc=1, 
            cmap='jet',
            titles=['original image', 'GradCAM++, label %s' % int(1)]
        )

    return figure, axis


In [ ]:
fig, axis = show_cam(video, grayscale_cam)